# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install async module
!uv pip install nest_asyncio

In [2]:
# Import the SDK
import syft_hub as sh
import nest_asyncio
import httpx
import logging
import os

In [3]:
# Import client
from syft_hub import Client
from syft_hub.utils.logger import get_logger

In [4]:
# define utilities
nest_asyncio.apply()
logger = get_logger()
logging.getLogger('syft_hub').setLevel(logging.INFO)
logger.success("Logger initialized successfully!")

2025-09-05 11:50:41 - syft_hub - SUCCESS - Logger initialized successfully!


In [5]:
# Initialize client
client = Client()

2025-09-05 11:50:41 - syft_hub.core.config - INFO - Using config credentials at path: /Users/callisezenwaka/.syftbox/config.json
2025-09-05 11:50:41 - syft_hub.main - INFO - Found existing accounting credentials for callis@openmined.org
2025-09-05 11:50:41 - syft_hub.main - INFO - Client initialized for callis@openmined.org


## Basic usage

In [6]:
# inline service usage - , "bob@example.com/wiki"
inline_result = client.pipeline(
    data_sources=["callis@openmined.org/carl-model", "callis@openmined.org/carl-free"], 
    synthesizer="callis@openmined.org/carl-model"
).run(messages=[{"role": "user", "content": "What is Python?"}])

2025-09-05 11:52:48 - syft_hub.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

                                                                                

2025-09-05 11:52:48 - syft_hub.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 11:52:48 - syft_hub.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 11:52:48 - syft_hub.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received



ValidationError: VALIDATION_ERROR: All data source searches failed

In [ ]:
# show response
logger.info(f"Pipeline result:\n\n {inline_result}")

In [ ]:
# Object-Oriented service usage
pipeline = client.create_pipeline()
pipeline.add_source("callis@openmined.org/sota-free", topK=8)
pipeline.add_source("callis@openmined.org/carl-model", topK=8)
pipeline.set_synthesizer("callis@openmined.org/sota-free", temperature=0.7)  
oop_result = pipeline.run(messages=[{"role": "user", "content": "What is Python?"}])

In [ ]:
# show response
logger.info(f"Pipeline result:\n\n {oop_result}")

## Advanced usage

In [6]:
# Load some services as objects first
gpt = client.load("callis@openmined.org/carl-model") # paid
gist = client.load("callis@openmined.org/sota-free") # free
wiki = client.load("callis@openmined.org/carl-free") # free
docs = client.load("callis@openmined.org/carl-claude") # free

In [7]:
# Demo mixed input types
mixed_result = client.pipeline_mixed(
    data_sources=[
        "callis@openmined.org/sota-free",           # String
        wiki,                                       # Service object
        {"name": "callis@openmined.org/carl-claude", "topK": 5},  # Dict with string
        {"name": gpt, "topK": 8}                   # Dict with Service object
    ],
    synthesizer=gpt  # Service object as synthesizer
).run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             

                                                                                

2025-09-05 08:25:35 - syft_hub.core.pipeline - ERROR - Search failed for callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 08:25:35 - syft_hub.core.pipeline - WARNING - Search failed for source callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             



In [8]:
logger.info(f"Mixed pipeline result:\n\n {mixed_result}")

2025-09-05 08:25:46 - syft_hub - INFO - Mixed pipeline result:

 Pyhton is a programming language that is widely used for data analysis, scientific computing, and machine learning. It was developed by Guido Van Helsdingen in the early 1990s and first released in 1991 as Python Implementation in C for the Apple IIe. Python has since become one of the most popular programming languages with over 37 million downloads, making it among the top ten most-downloaded software programs worldwide, according to StatCounter (as of December 2021).

Python is known for its ease of use and flexibility, which allows users to quickly build complex applications without requiring extensive programming knowledge. It has a vast community of developers who create libraries, extensions, and packages that support various areas of Python development, including data analysis, machine learning, artificial intelligence (AI), scientific computing, and more.

Python is widely used in fields such as finance, healthca

In [ ]:
# Load some services as objects first
carl_model = client.load("callis@openmined.org/carl-model") # paid
sota_free = client.load("callis@openmined.org/sota-free") # free
carl_free = client.load("callis@openmined.org/carl-free") # free
carl_claude = client.load("callis@openmined.org/carl-claude") # free

In [12]:
# Demo with pure Service objects
service_result = client.pipeline_mixed(
    data_sources=[sota_free, carl_free],
    synthesizer=sota_free
).run(messages=[{"role": "user", "content": "What is machine learning?"}])

✓ Response received                                                             



In [13]:
logger.info(f"Service objects pipeline result:\n\n {service_result}")

2025-09-05 01:53:57 - syft_nsai_sdk - INFO - Service objects pipeline result:

 Machine learning is a subfield of artificial intelligence that involves the development and application of algorithms to learn from data in order to make predictions or decisions. It combines aspects of statistics, computer science, and economics, and has applications across various fields such as finance, healthcare, transportation, and retail. Machine learning algorithms rely on large datasets to learn patterns and make predictions, often using machine learning techniques like decision trees, neural networks, and random forests. By leveraging this knowledge, machine learning algorithms can help solve complex problems, making it a powerful tool in many industries.


In [16]:
# Demo Service objects with different parameters
param_result = client.pipeline_mixed(
    data_sources=[
        {"name": carl_free, "topK": 10},
        {"name": "callis@openmined.org/carl-claude", "topK": 3}
    ],
    synthesizer={"name": carl_model, "temperature": 0.9}
).run(messages=[{"role": "user", "content": "Explain neural networks"}])

2025-09-05 01:56:56 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-claude: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 01:56:56 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-claude: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             



In [17]:
logger.info(f"Parameters pipeline result:\n\n {param_result}")

2025-09-05 01:57:03 - syft_nsai_sdk - INFO - Parameters pipeline result:

 Neural networks are a type of artificial intelligence (AI) agent that can be trained to simulate and analyze complex patterns. These patterns may come from real-world data or from previous experiences in the form of patterns learned through trial and error. Neural networks consist of several layers of neurons, or nodes, connected together by weighted connections. Each layer of neurons has different weights, which determine how much input is passed to that neuron and how it affects the next layer of neurons.

Neural networks have been applied in a wide variety of applications, including natural language processing (NLP), image recognition, and speech recognition. However, the development of these neural networks requires the use of data and a well-defined mathematical framework that supports the training process. One such framework is backpropagation, which involves updating each node's weight to minimize an erro